# Encontro 13: Medidas de Centralidade
### Por Isabella Oliveira
Importando a biblioteca:

In [59]:
import sys
sys.path.append('..')

from random import choice
from itertools import permutations

import pandas as pd
import networkx as nx

import socnet as sn
import numpy as np

Definindo as configurações da biblioteca socnet

In [60]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

sn.graph_width = 800
sn.graph_height = 345

sn.node_label_position = 'top center'


Rodando o grafo

In [61]:
g = sn.load_graph('Renaissance.gml', has_pos=True)

sn.show_graph(g, nlab=True)

Implementação das funções já disponibilizadas

In [62]:
def set_geodesic_successors(g, s, t):
    for n in g.nodes:
        g.nodes[n]['geodesic_successors'] = set()

    for p in nx.all_shortest_paths(g, s, t):
        for i in range(len(p) - 1):
            g.nodes[p[i]]['geodesic_successors'].add(p[i + 1])

In [63]:
# Pense que o atributo 'passages' abaixo indica quantas
# vezes um fluxo já passou por um nó ou por uma aresta.

def random_geodesic_successor(g, n):
    return choice([m for m in g.nodes[n]['geodesic_successors']])

def random_path_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.nodes[m]['passages'] == 0])

def random_trail_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.edges[n, m]['passages'] == 0])

def random_walk_successor(g, n):
    return choice([m for m in g.neighbors(n)])

Modificação na função disponibilizada para que aceite mais tipos de trajetória além de geodésica e para que aceite também a difusão por duplicação

In [64]:
def simulate_single_flow(g, s, t, option, isTransf):
    # Inicializa o atributo 'passages' de cada nó.
    for n in g.nodes:
        g.nodes[n]['passages'] = 0
    g.nodes[s]['passages'] = 1

    # Inicializa o atributo 'passages' de cada aresta.
    for n, m in g.edges:
        g.edges[n, m]['passages'] = 0

    # Inicializa s como o único dono do insumo.
    for n in g.nodes:
        g.nodes[n]['owner'] = False
    g.nodes[s]['owner'] = True

    # Simula o fluxo, contando o número total de passos.

    steps = 0

    while True:
        # O conjunto reached representa todos os nós
        # que o fluxo consegue alcançar no passo atual.
        reached = set()

        # Verifica cada um dos donos atuais do insumo.

        owners = [n for n in g.nodes if g.nodes[n]['owner']]

        for n in owners:

            try:
                # Seleciona o tipo de trajetória passado nos parâmetros = geodesic, path, trail or walk
                m = option(g, n)
                    
            except IndexError:
                continue

            # Deixa de ser dono do insumo caso seja transferência. Caso não, continua sendo
            g.nodes[n]['owner'] = not(isTransf)

            # Incrementa o atributo 'passages' do nó.
            g.nodes[m]['passages'] += 1

            # Incrementa o atributo 'passages' da aresta.
            g.edges[n, m]['passages'] += 1

            # Registra que consegue alcançar esse nó.
            reached.add(m)

        # Todo nó alcançado passa a ser dono do insumo.

        for n in reached:
            g.nodes[n]['owner'] = True

        # Isso conclui o passo atual da simulação.
        steps += 1

        # Se o passo alcançou t, chegamos ao fim da simulação.
        # Ela foi bem-sucedida: devolvemos o número de passos.
        if t in reached:
            return steps

        # Se o passo não alcançou ninguém, chegamos ao fim da
        # simulação. Ela não foi bem-sucedida: devolvemos -1.
        if not reached:
            return -1

In [65]:
def simulate_successful_flow(g, s, t, option, isTransf):
    set_geodesic_successors(g, s, t)

    while True:
        steps = simulate_single_flow(g, s, t, option, isTransf)

        if steps != -1:
            return steps

In [66]:
def simulate_all_flows(g, option, isTransf):
    for n in g.nodes:
        g.nodes[n]['closeness'] = 0
        g.nodes[n]['betweenness'] = 0

    for s, t in permutations(g.nodes, 2):
        steps = simulate_successful_flow(g, s, t, option, isTransf)

        g.nodes[s]['closeness'] += steps
        for n in g.nodes:
            if n != s and n != t:
                g.nodes[n]['betweenness'] += g.nodes[n]['passages']

    # Normalizações necessárias para comparar com os
    # resultados analíticos. Não é preciso entender.
    for n in g.nodes:
        g.nodes[n]['closeness'] = (g.number_of_nodes() - 1) / g.nodes[n]['closeness']
        g.nodes[n]['betweenness'] /= (g.number_of_nodes() - 1) * (g.number_of_nodes() - 2)

No código abaixo há uma função que retorna a lista de closeness, betweenness, e a média dos dois em 100 simulações, para cada família.

In [67]:
def simulacoes(g, option, isTransf):
    TIMES = 100

    for n in g.nodes:
        
        g.nodes[n]['lista_closeness'] = []
        g.nodes[n]['lista_betweenness'] = []
        g.nodes[n]['mean_closeness'] = 0
        g.nodes[n]['mean_betweenness'] = 0

    for _ in range(TIMES):
        simulate_all_flows(g, option, isTransf)

        for n in g.nodes:
            g.nodes[n]['lista_closeness'].append(g.nodes[n]['closeness'])
            g.nodes[n]['lista_betweenness'].append(g.nodes[n]['betweenness'])
            g.nodes[n]['mean_closeness'] += g.nodes[n]['closeness']
            g.nodes[n]['mean_betweenness'] += g.nodes[n]['betweenness']
            
    for n in g.nodes:
        g.nodes[n]['mean_closeness'] /= TIMES
        g.nodes[n]['mean_betweenness'] /= TIMES
        
    return {n: (g.nodes[n]['lista_closeness'], g.nodes[n]['lista_betweenness'], g.nodes[n]['mean_closeness'], g.nodes[n]['mean_betweenness']) for n in g.nodes}

Abaixo há as 100 simulações dos closeness e betweenness de todas as famílias para todas as combinações de trajetória e difusão, para todas as famílias.

In [68]:
geoTrue = simulacoes(g, random_geodesic_successor, True)
geoFalse = simulacoes(g, random_geodesic_successor, False)
pathTrue = simulacoes(g, random_path_successor, True)
pathFalse = simulacoes(g, random_path_successor, False)
trailTrue = simulacoes(g, random_trail_successor, True)
trailFalse = simulacoes(g, random_trail_successor, False)
walkTrue = simulacoes(g, random_walk_successor, True)
walkFalse = simulacoes(g, random_walk_successor, False)

#### Hipótese nula (a que queremos rejeitar)
Quando consideramos outros tipos de trajetória e outros tipos de difusão, os nós com maior closeness simulado e betweenness simulado são necessariamente os nós com maior closeness e betweenness segundo as fórmulas clássicas. (que correspondem ao uso de geodésica e transferência na simulação)

#### Hipótese alternativa (a que queremos provar)
Quando consideramos outros tipos de trajetória e outros tipos de difusão, os nós com maior closeness simulado e betweenness simulado NÃO são necessariamente os nós com maior closeness e betweenness segundo as fórmulas clássicas. (que correspondem ao uso de geodésica e transferência na simulação)

A função abaixo foi retirada do link: https://pt.stackoverflow.com/questions/176243/como-limitar-números-decimais-em-python apenas para truncar os valores decimais com muitas casas após a vírgula e facilitar a visualização das tabelas que serão geradas a seguir.
Foi feita uma alteração para considerar os resultados nan.

In [69]:
def truncate(f, n):
    '''Truncates/pads a float f to n decimal places without rounding'''
    s = '{}'.format(f)
    if np.isnan(f):
        return f
    elif 'e' in s or 'E' in s:
        return '{0:.{1}f}'.format(f, n)
    
    i, p, d = s.partition('.')
    return '.'.join([i, (d+'0'*n)[:n]])

Abaixo são armazenados os p-valores oriundos dos testes t-student realizados tanto para o closenness quanto para o betweenness de cada família, para cada tipo de trajetória combinado a cada difusão.

In [70]:
from scipy import stats
a = 0
for n in g.nodes:
    g.nodes[n]['pvalor_closeness'] = []
    g.nodes[n]['pvalor_betweenness'] = []
    #Comparando default (geodésica transferência) com geodésica duplicação
    g.nodes[n]['pvalor_closeness'].append(truncate(stats.ttest_ind(geoTrue[n][0],geoFalse[n][0])[1], 3))
    g.nodes[n]['pvalor_betweenness'].append(truncate(stats.ttest_ind(geoTrue[n][1],geoFalse[n][1])[1], 3))
    #Comparando default (geodésica transferência) com path transferência
    g.nodes[n]['pvalor_closeness'].append(truncate(stats.ttest_ind(geoTrue[n][0],pathTrue[n][0])[1], 3))
    g.nodes[n]['pvalor_betweenness'].append(truncate(stats.ttest_ind(geoTrue[n][1],pathTrue[n][1])[1], 3))
    #Comparando default (geodésica transferência) com path duplicação
    g.nodes[n]['pvalor_closeness'].append(truncate(stats.ttest_ind(geoTrue[n][0],pathFalse[n][0])[1], 3))
    g.nodes[n]['pvalor_betweenness'].append(truncate(stats.ttest_ind(geoTrue[n][1],pathFalse[n][1])[1], 3))
    #Comparando default (geodésica transferência) com trail transferência
    g.nodes[n]['pvalor_closeness'].append(truncate(stats.ttest_ind(geoTrue[n][0],trailTrue[n][0])[1], 3))
    g.nodes[n]['pvalor_betweenness'].append(truncate(stats.ttest_ind(geoTrue[n][1],trailTrue[n][1])[1], 3))
    #Comparando default (geodésica transferência) com trail duplicação
    g.nodes[n]['pvalor_closeness'].append(truncate(stats.ttest_ind(geoTrue[n][0], trailFalse[n][0])[1], 3))
    g.nodes[n]['pvalor_betweenness'].append(truncate(stats.ttest_ind(geoTrue[n][1],trailFalse[n][1])[1], 3))
    #Comparando default (geodésica transferência) com walk transferência
    g.nodes[n]['pvalor_closeness'].append(truncate(stats.ttest_ind(geoTrue[n][0],walkTrue[n][0])[1], 3))
    g.nodes[n]['pvalor_betweenness'].append(truncate(stats.ttest_ind(geoTrue[n][1],walkTrue[n][1])[1], 3))
    #Comparando default (geodésica transferência) com walk duplicação
    g.nodes[n]['pvalor_closeness'].append(truncate(stats.ttest_ind(geoTrue[n][0],walkFalse[n][0])[1], 3))
    g.nodes[n]['pvalor_betweenness'].append(truncate(stats.ttest_ind(geoTrue[n][1],walkFalse[n][1])[1], 3))

In [71]:
#Exemplo do funcionamento do código:
#g.nodes[família]['pvalor_closeness'][qual comparação]
#print(g.nodes[4]['pvalor_closeness'][5])

Abaixo há a comparação dos closeness simulados para todas as famílias e combinações de trajetórias e difusões.

In [72]:
#Comparação dos Closeness
pd.DataFrame({
    ' Família': [g.nodes[n]['label'] for n in g.nodes],
    'Geodésica Duplicação': [g.nodes[n]['pvalor_closeness'][0] for n in g.nodes],
    'Caminho Transferência': [g.nodes[n]['pvalor_closeness'][1] for n in g.nodes],
    'Caminho Duplicação': [g.nodes[n]['pvalor_closeness'][2] for n in g.nodes],
    'Trilha Transferência': [g.nodes[n]['pvalor_closeness'][3] for n in g.nodes],
    'Trilha Duplicação': [g.nodes[n]['pvalor_closeness'][4] for n in g.nodes],
    'Passeio Transferência': [g.nodes[n]['pvalor_closeness'][5] for n in g.nodes],
    'Passeio Duplicação': [g.nodes[n]['pvalor_closeness'][6] for n in g.nodes],
})

,Família,Caminho Duplicação,Caminho Transferência,Geodésica Duplicação,Passeio Duplicação,Passeio Transferência,Trilha Duplicação,Trilha Transferência
0,ginori,0.000,0.000,1.000,0.000,0.000,0.000,0.000
1,lambertes,0.000,0.000,1.000,0.000,0.000,0.000,0.000
2,albizzi,0.000,0.000,1.000,0.000,0.000,0.000,0.000
3,guadagni,0.000,0.000,1.000,0.000,0.000,0.000,0.000
4,pazzi,0.000,0.000,1.000,0.000,0.000,0.000,0.000
5,salviati,0.000,0.000,1.000,0.000,0.000,0.000,0.000
6,medici,0.000,0.000,1.000,0.000,0.000,0.000,0.000
7,tornabuon,0.000,0.000,1.000,0.000,0.000,0.000,0.000
8,bischeri,0.000,0.000,1.000,0.000,0.000,0.000,0.000
9,ridolfi,0.000,0.000,1.000,0.000,0.000,0.000,0.000


Abaixo há a comparação dos betweenness simulados para todas as famílias e combinações de trajetórias e difusões.

In [73]:
#Comparação dos Betweenness
pd.DataFrame({
    ' Família': [g.nodes[n]['label'] for n in g.nodes],
    'Geodésica Duplicação': [g.nodes[n]['pvalor_betweenness'][0] for n in g.nodes],
    'Caminho Transferência': [g.nodes[n]['pvalor_betweenness'][1] for n in g.nodes],
    'Caminho Duplicação': [g.nodes[n]['pvalor_betweenness'][2] for n in g.nodes],
    'Trilha Transferência': [g.nodes[n]['pvalor_betweenness'][3] for n in g.nodes],
    'Trilha Duplicação': [g.nodes[n]['pvalor_betweenness'][4] for n in g.nodes],
    'Passeio Transferência': [g.nodes[n]['pvalor_betweenness'][5] for n in g.nodes],
    'Passeio Duplicação': [g.nodes[n]['pvalor_betweenness'][6] for n in g.nodes],
})

,Família,Caminho Duplicação,Caminho Transferência,Geodésica Duplicação,Passeio Duplicação,Passeio Transferência,Trilha Duplicação,Trilha Transferência
0,ginori,0.000,NaN,NaN,0.000,0.000,0.000,NaN
1,lambertes,0.000,NaN,NaN,0.000,0.000,0.000,NaN
2,albizzi,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,guadagni,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,pazzi,0.000,NaN,NaN,0.000,0.000,0.000,NaN
5,salviati,0.000,1.000,0.000,0.000,0.000,0.000,1.000
6,medici,0.000,0.000,0.000,0.000,0.000,0.000,0.000
7,tornabuon,0.000,0.000,0.000,0.000,0.000,0.000,0.000
8,bischeri,0.000,0.000,0.000,0.000,0.000,0.000,0.000
9,ridolfi,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [74]:
#sort_values(): to sort pandas data frame by one or more columns
#sort_index(): to sort pandas data frame by row index
#describe() Function gives the mean, std and IQR values. It excludes character column and calculate summary statistics only for numeric columns

Abaixo há a tentativa de entender o motivo que causou os resultados "Not a number" nas tabelas de p-valor geradas.

In [75]:
#Tratamento Nan betweenness - famílias 0, 1, 4, 10. Caminhos geoFalse, pathTrue, trailTrue
listinha = [0, 1, 4, 10]
for i in listinha:
    print("\n Valores do betweenness simulado da família", i, "no tipo geodésica com duplicação:")
    print(geoFalse[i][1])
    print("\n Valores do betweenness simulado da família", i, "no tipo path com transferência:")
    print(pathTrue[i][1])
    print("\n Valores do betweenness simulado da família", i, "no tipo trail com transferência:")
    print(trailTrue[i][1])
print("\n Todos os valores obtidos foram 0 e por conta disso o teste t-student não pode ser calculado e o p-valor deu Nan")
#Tudo 0, por isso o p-valor deu Nan.

#Tratamento Nan closenness - família 13. Trajetória geoFalse
print("\n Valores do closeness simulado da família Barbadori no tipo geodésica com duplicação:")
print(geoFalse[13][0])
print("\n Valores do closeness simulado da família Barbadori no tipo geodésica com transferência:")
print(geoTrue[13][0])
print("\n P-valor do closeness da família Barbadori para geodésica com duplicação")
print(g.nodes[13]['pvalor_closeness'][0])
print("\n t-student do closeness da família Barbadori para geodésica com duplicação")
print(stats.ttest_ind(geoTrue[13][0],geoFalse[13][0])[0])
print("\n Sem motivo aparente para o p-valor da família Barbadori ser Nan para geodésica com duplicação já que todas as simulações deram valores maiores que 0 e menores que 1.")
#Sem motivo aparente para o p-valor ser Nan já que todas as simulações deram valores maiores que 0 e menores que 1.


 Valores do betweenness simulado da família 0 no tipo geodésica com duplicação:
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

 Valores do betweenness simulado da família 0 no tipo path com transferência:
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [76]:
#Conferindo centralidades e p-valores da família salviati, já que obteve p-valores com iguais a 1 para path e trail com transferência.
print("p-valor para caminho com transferência:", g.nodes[5]['pvalor_betweenness'][1]) #path transf
print("p-valor para trilha com transferência:", g.nodes[5]['pvalor_betweenness'][3]) #trail transf
print("Média do betweenness para geodésica com transferência:", geoTrue[5][3]) #betweenness geodesica com transferencia
print("Média do betweenness para caminho com transferência:", pathTrue[5][3]) #betweenness path com transferencia
print("Média do betweenness para trilha com transferência:", trailTrue[5][3]) #betweenness trail com transferencia
print("\nA média dos betweenness simulados para os trajetórias path e trail, ambas com transferência, deram a mesma que a média do betweenness simulado para geodésica com transferência para a família Salviati, por isso o p-valor para o betweenness nesses casos resultou em 1.")

p-valor para caminho com transferência: 1.000
p-valor para trilha com transferência: 1.000
Média do betweenness para geodésica com transferência: 0.1428571428571427
Média do betweenness para caminho com transferência: 0.1428571428571427
Média do betweenness para trilha com transferência: 0.1428571428571427

A média dos betweenness simulados para os trajetórias path e trail, ambas com transferência, deram a mesma que a média do betweenness simulado para geodésica com transferência para a família Salviati, por isso o p-valor para o betweenness nesses casos resultou em 1.


In [77]:
bet = pd.DataFrame({
        'Família': [g.nodes[n]['label'] for n in g.nodes],
        'média closeness': [geoTrue[n][2] for n in g.nodes],
        'média betweenness': [geoTrue[n][3] for n in g.nodes]})

sortc = bet.sort_values(by=['média closeness'], ascending=False)
sortb = bet.sort_values(by=['média betweenness'], ascending=False)

É interessante comparar a posição de cada família em um ranking tanto de closeness quanto de betweenness para todas as combinações de trajetória e difusão afim de descobrir se a importância das famílias muda ou não de acordo com a trajetória e a difusão escolhida.

In [78]:
def sortingcb(compara):
    bet = pd.DataFrame({
            'Família': [g.nodes[n]['label'] for n in g.nodes],
            'média closeness': [compara[n][2] for n in g.nodes],
            'média betweenness': [compara[n][3] for n in g.nodes]})
    
    return bet.sort_values(by=['média closeness'], ascending=False), bet.sort_values(by=['média betweenness'], ascending=False)

As células abaixo ordenam de forma decrescente o closeness e o betweenness simulados de todas as famílias em todos os tipos de trajetórias combinados com as difusões. Isso servirá para gerar a tabela de comparação de posições de importância das famílias.

In [79]:
CGT = []
CGF = []
CPT = []
CPF = []
CTT = [] 
CTF = []
CWT = []
CWF = []

lista = [CGT, CGF, CPT, CPF, CTT, CTF, CWT, CWF]
caminhos = (geoTrue, geoFalse, pathTrue, pathFalse, trailTrue, trailFalse, walkTrue, walkFalse)
for n in g.nodes:
    for i in range(len(lista)):
        lista[i].append((g.nodes[n]['label'], caminhos[i][n][2]))

CGT = sorted(CGT, key=lambda x: x[1], reverse=True) 
CGF = sorted(CGF, key=lambda x: x[1], reverse=True) 
CPT = sorted(CPT, key=lambda x: x[1], reverse=True) 
CPF = sorted(CPF, key=lambda x: x[1], reverse=True) 
CTT = sorted(CTT, key=lambda x: x[1], reverse=True) 
CTF = sorted(CTF, key=lambda x: x[1], reverse=True) 
CWT = sorted(CWT, key=lambda x: x[1], reverse=True) 
CWF = sorted(CWF, key=lambda x: x[1], reverse=True)
lista = [CGT, CGF, CPT, CPF, CTT, CTF, CWT, CWF]

for k in lista:
    for n in range(len(k)):
        k[n] = (k[n][0], n)
        
CGT = sorted(CGT, key=lambda x: x[0]) 
CGF = sorted(CGF, key=lambda x: x[0]) 
CPT = sorted(CPT, key=lambda x: x[0]) 
CPF = sorted(CPF, key=lambda x: x[0]) 
CTT = sorted(CTT, key=lambda x: x[0]) 
CTF = sorted(CTF, key=lambda x: x[0]) 
CWT = sorted(CWT, key=lambda x: x[0]) 
CWF = sorted(CWF, key=lambda x: x[0])

In [80]:
BGT = []
BGF = []
BPT = []
BPF = []
BTT = [] 
BTF = []
BWT = []
BWF = []

lista = [BGT, BGF, BPT, BPF, BTT, BTF, BWT, BWF]
caminhos = (geoTrue, geoFalse, pathTrue, pathFalse, trailTrue, trailFalse, walkTrue, walkFalse)
for n in g.nodes:
    for i in range(len(lista)):
        lista[i].append((g.nodes[n]['label'], caminhos[i][n][3]))

BGT = sorted(BGT, key=lambda x: x[1], reverse=True) 
BGF = sorted(BGF, key=lambda x: x[1], reverse=True) 
BPT = sorted(BPT, key=lambda x: x[1], reverse=True) 
BPF = sorted(BPF, key=lambda x: x[1], reverse=True) 
BTT = sorted(BTT, key=lambda x: x[1], reverse=True) 
BTF = sorted(BTF, key=lambda x: x[1], reverse=True) 
BWT = sorted(BWT, key=lambda x: x[1], reverse=True) 
BWF = sorted(BWF, key=lambda x: x[1], reverse=True)
lista = [BGT, BGF, BPT, BPF, BTT, BTF, BWT, BWF]

for k in lista:
    for n in range(len(k)):
        k[n] = (k[n][0], n)
        
BGT = sorted(BGT, key=lambda x: x[0]) 
BGF = sorted(BGF, key=lambda x: x[0]) 
BPT = sorted(BPT, key=lambda x: x[0]) 
BPF = sorted(BPF, key=lambda x: x[0]) 
BTT = sorted(BTT, key=lambda x: x[0]) 
BTF = sorted(BTF, key=lambda x: x[0]) 
BWT = sorted(BWT, key=lambda x: x[0]) 
BWF = sorted(BWF, key=lambda x: x[0])

In [81]:
familias = sorted(g.nodes[n]['label'] for n in g.nodes)
print("Comparação entre a posição dos closeness simulados ")
pd.DataFrame({
    ' Família': [familias[n] for n in g.nodes],
    'Geodésica Transfer': [CGT[n][1] for n in g.nodes],
    'Geodésica Duplica': [CGF[n][1] for n in g.nodes],
    'Caminho Transfer': [CPT[n][1] for n in g.nodes],
    'Caminho Duplica': [CPF[n][1] for n in g.nodes],
    'Trilha Transfer': [CTT[n][1] for n in g.nodes],
    'Trilha Duplica': [CTF[n][1] for n in g.nodes],
    'Passeio Transfer': [CWT[n][1] for n in g.nodes],
    'Passeio Duplica': [CWF[n][1] for n in g.nodes],
})

Comparação entre a posição dos closeness simulados 


,Família,Caminho Duplica,Caminho Transfer,Geodésica Duplica,Geodésica Transfer,Passeio Duplica,Passeio Transfer,Trilha Duplica,Trilha Transfer
0,acciaiuol,10,7,10,10,9,2,10,11
1,albizzi,8,3,1,1,5,14,7,2
2,barbadori,2,9,5,5,1,4,3,3
3,bischeri,4,8,7,7,6,9,5,8
4,castellan,7,5,9,9,8,3,8,9
5,ginori,12,13,12,12,13,8,12,12
6,guadagni,6,1,4,4,4,7,6,1
7,lambertes,11,12,13,13,12,0,11,13
8,medici,3,0,0,0,2,13,2,0
9,pazzi,14,14,14,14,14,1,14,14


In [82]:
print("Comparação entre a posição dos betweenness simulados ")
pd.DataFrame({
    ' Família': [familias[n] for n in g.nodes],
    'Geodésica Transfer': [BGT[n][1] for n in g.nodes],
    'Geodésica Duplica': [BGF[n][1] for n in g.nodes],
    'Caminho Transfer': [BPT[n][1] for n in g.nodes],
    'Caminho Duplica': [BPF[n][1] for n in g.nodes],
    'Trilha Transfer': [BTT[n][1] for n in g.nodes],
    'Trilha Duplica': [BTF[n][1] for n in g.nodes],
    'Passeio Transfer': [BWT[n][1] for n in g.nodes],
    'Passeio Duplica': [BWF[n][1] for n in g.nodes],
})

Comparação entre a posição dos betweenness simulados 


,Família,Caminho Duplica,Caminho Transfer,Geodésica Duplica,Geodésica Transfer,Passeio Duplica,Passeio Transfer,Trilha Duplica,Trilha Transfer
0,acciaiuol,12,14,14,14,14,11,13,14
1,albizzi,7,9,2,2,3,5,7,8
2,barbadori,8,8,4,5,10,8,8,7
3,bischeri,5,3,5,4,4,4,6,3
4,castellan,6,2,8,8,2,7,5,2
5,ginori,13,11,11,11,11,13,12,11
6,guadagni,1,1,1,1,1,1,3,1
7,lambertes,11,12,12,12,13,12,11,12
8,medici,0,0,0,0,0,0,0,0
9,pazzi,14,13,13,13,12,14,14,13
